In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path

In [2]:
df = pd.read_csv(Path('./data/20200810.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],Rank your movie choices. You don't have to do a full ranking. [Sixth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice],What theme should we explore next week? [Seventh Choice]
0,8/5/2020 17:49:40,The Great Silence,Hands Off the Loot!,Pumping Iron II: The Women,Meathead Goes Hog Wild,West Side Story,NaN,CATHOLIC SEX GUILT,No Theme/Free-for-all,PARTYING GONE WRONG,SET PRIMARILY ON WATER,DUDE HANGS DONG,"OOPS, ALL CHRISOPHER MELONI",DISGUISES
1,8/5/2020 17:55:52,The Great Silence,Hands Off the Loot!,Meathead Goes Hog Wild,Pumping Iron,Pumping Iron II: The Women,West Side Story,SET PRIMARILY ON WATER,"OOPS, ALL CHRISOPHER MELONI",PARTYING GONE WRONG,DUDE HANGS DONG,DISGUISES,CATHOLIC SEX GUILT,No Theme/Free-for-all
2,8/5/2020 18:12:21,The Great Silence,Pumping Iron,Meathead Goes Hog Wild,Pumping Iron II: The Women,Hands Off the Loot!,West Side Story,PARTYING GONE WRONG,CATHOLIC SEX GUILT,DISGUISES,No Theme/Free-for-all,"OOPS, ALL CHRISOPHER MELONI",SET PRIMARILY ON WATER,DUDE HANGS DONG
3,8/5/2020 20:14:18,The Great Silence,Pumping Iron,Meathead Goes Hog Wild,Hands Off the Loot!,NaN,NaN,DISGUISES,PARTYING GONE WRONG,SET PRIMARILY ON WATER,DUDE HANGS DONG,"OOPS, ALL CHRISOPHER MELONI",CATHOLIC SEX GUILT,NaN
4,8/5/2020 21:34:40,Meathead Goes Hog Wild,The Great Silence,Hands Off the Loot!,Pumping Iron,Pumping Iron II: The Women,West Side Story,"OOPS, ALL CHRISOPHER MELONI",PARTYING GONE WRONG,SET PRIMARILY ON WATER,CATHOLIC SEX GUILT,DISGUISES,DUDE HANGS DONG,No Theme/Free-for-all
5,8/6/2020 8:03:24,The Great Silence,Hands Off the Loot!,Meathead Goes Hog Wild,Pumping Iron II: The Women,Pumping Iron,West Side Story,CATHOLIC SEX GUILT,PARTYING GONE WRONG,SET PRIMARILY ON WATER,DISGUISES,DUDE HANGS DONG,"OOPS, ALL CHRISOPHER MELONI",No Theme/Free-for-all
6,8/8/2020 9:07:21,Meathead Goes Hog Wild,The Great Silence,Hands Off the Loot!,Pumping Iron II: The Women,Pumping Iron,West Side Story,CATHOLIC SEX GUILT,"OOPS, ALL CHRISOPHER MELONI",SET PRIMARILY ON WATER,DUDE HANGS DONG,DISGUISES,PARTYING GONE WRONG,No Theme/Free-for-all
7,8/8/2020 9:36:13,Meathead Goes Hog Wild,The Great Silence,Pumping Iron,West Side Story,Pumping Iron II: The Women,Hands Off the Loot!,"OOPS, ALL CHRISOPHER MELONI",DISGUISES,SET PRIMARILY ON WATER,PARTYING GONE WRONG,DUDE HANGS DONG,CATHOLIC SEX GUILT,No Theme/Free-for-all
8,8/8/2020 16:05:13,West Side Story,The Great Silence,Meathead Goes Hog Wild,Pumping Iron II: The Women,NaN,NaN,"OOPS, ALL CHRISOPHER MELONI",DUDE HANGS DONG,CATHOLIC SEX GUILT,NaN,NaN,NaN,NaN


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['The Great Silence', 'Hands Off the Loot!',
       'Meathead Goes Hog Wild', 'Pumping Iron',
       'Pumping Iron II: The Women', 'West Side Story'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'The Great Silence': <Candidate('The Great Silence')>,
 'Hands Off the Loot!': <Candidate('Hands Off the Loot!')>,
 'Meathead Goes Hog Wild': <Candidate('Meathead Goes Hog Wild')>,
 'Pumping Iron': <Candidate('Pumping Iron')>,
 'Pumping Iron II: The Women': <Candidate('Pumping Iron II: The Women')>,
 'West Side Story': <Candidate('West Side Story')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(The Great Silence, Hands Off the Loot!, Pumping Iron II: The Women, Meathead Goes Hog Wild, West Side Story)>,
 <Ballot(The Great Silence, Hands Off the Loot!, Meathead Goes Hog Wild, Pumping Iron, Pumping Iron II: The Women, West Side Story)>,
 <Ballot(The Great Silence, Pumping Iron, Meathead Goes Hog Wild, Pumping Iron II: The Women, Hands Off the Loot!, West Side Story)>,
 <Ballot(The Great Silence, Pumping Iron, Meathead Goes Hog Wild, Hands Off the Loot!)>,
 <Ballot(Meathead Goes Hog Wild, The Great Silence, Hands Off the Loot!, Pumping Iron, Pumping Iron II: The Women, West Side Story)>,
 <Ballot(The Great Silence, Hands Off the Loot!, Meathead Goes Hog Wild, Pumping Iron II: The Women, Pumping Iron, West Side Story)>,
 <Ballot(Meathead Goes Hog Wild, The Great Silence, Hands Off the Loot!, Pumping Iron II: The Women, Pumping Iron, West Side Story)>,
 <Ballot(Meathead Goes Hog Wild, The Great Silence, Pumping Iron, West Side Story, Pumping Iron II: The Women, Hands Off 

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

FINAL RESULT
Candidate                     Votes  Status
--------------------------  -------  --------
The Great Silence                 5  Elected
Meathead Goes Hog Wild            3  Rejected
West Side Story                   1  Rejected
Hands Off the Loot!               0  Rejected
Pumping Iron                      0  Rejected
Pumping Iron II: The Women        0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                      Votes  Status
---------------------------  -------  --------
OOPS, ALL CHRISOPHER MELONI        3  Hopeful
CATHOLIC SEX GUILT                 3  Hopeful
DISGUISES                          1  Hopeful
PARTYING GONE WRONG                1  Rejected
SET PRIMARILY ON WATER             1  Rejected
DUDE HANGS DONG                    0  Rejected
No Theme/Free-for-all              0  Rejected

ROUND 2
Candidate                      Votes  Status
---------------------------  -------  --------
OOPS, ALL CHRISOPHER MELONI        4  Hopeful
CATHOLIC SEX GUILT                 4  Hopeful
DISGUISES                          1  Rejected
PARTYING GONE WRONG                0  Rejected
SET PRIMARILY ON WATER             0  Rejected
DUDE HANGS DONG                    0  Rejected
No Theme/Free-for-all              0  Rejected

FINAL RESULT
Candidate                      Votes  Status
---------------------------  -------  --------
OOPS, ALL CHRISOPHER MELONI        5  El